# Setup

In [1]:
!pip install -qU langchain
!pip install -qU langchain-google-genai
!pip install -qU langchain-huggingface
!pip install -qU langchain-qdrant
!pip install -qU langchain-community
!pip install -qU langgraph
!pip install fastembed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 71.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.

In [8]:
import os
# from dotenv import load_dotenv
import pandas as pd
from pandas import DataFrame
import numpy as np
from tqdm import tqdm

# load_dotenv()

# for colab
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('gemini_api_key')
os.environ["HUGGINGFACE_API_KEY"] = userdata.get('huggingface_api_key')
os.environ["LANGSMITH_API_KEY"] = userdata.get('langsmith_api_key')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "nlp_project"

In [4]:
from huggingface_hub import login

login(token=os.getenv("HUGGINGFACE_API_KEY"))
print("Logged in successfully!")

Logged in successfully!


In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")

In [11]:
llm.invoke("hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run--b2d84605-1f56-4b37-9d4d-998e2ed6a016-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings

model_kwargs = {'trust_remote_code': True}
embeddings = HuggingFaceEmbeddings(model_name="Alibaba-NLP/gte-modernbert-base", model_kwargs=model_kwargs)

In [13]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="Alibaba-NLP/gte-reranker-modernbert-base")

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/598M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

## Loading data
- only include submissions with full score
- duplicate code (exact match) is not allowed for unknown/generated code
- duplicate code (only different variable names) is not allowed for human-written code

In [15]:
submissions = pd.read_csv("./data/submissions.csv")
submissions.groupby(["problem_id", "type"]).size().unstack(fill_value=0)

type,human,unknown
problem_id,,
05_List_11,2029,1304
08_Dict_22,2018,414


In [16]:
generated_submissions = pd.read_csv("./data/generated_submissions.csv")
generated_submissions.groupby("problem_id").size().reset_index(name="count")

,problem_id,count
0,05_List_11,62
1,08_Dict_22,378


In [17]:
submissions = pd.concat([submissions, generated_submissions], ignore_index=True)
submissions.groupby(["problem_id", "type"]).size().unstack(fill_value=0)

type,generated,human,unknown
problem_id,,,
05_List_11,62,2029,1304
08_Dict_22,378,2018,414


In [18]:
submissions

,id,author,problem_id,code,submission_id,score,time,type
0,1255109.0,6330139021,05_List_11,x = input()\na = [str(i) for i in range(10)]\n...,1255109.0,100.0,over 4 years ago,human
1,1257085.0,6330004021,05_List_11,"n=input()\nans=''\ndg=['0','1','2','3','4','5'...",1257085.0,100.0,over 4 years ago,human
2,1255424.0,6330015921,05_List_11,"a=input()\nb=[]\nc=""""\nfor ch in a:\n if""0""...",1255424.0,100.0,over 4 years ago,human
3,1047529.0,6230080521,05_List_11,"a=input().strip()\nn=['0','1','2','3','4','5',...",1047529.0,100.0,over 5 years ago,human
4,1253956.0,6231007021,05_List_11,"dg = ['0','1','2','3','4','5','6','7','8','9']...",1253956.0,100.0,over 4 years ago,human
...,...,...,...,...,...,...,...,...
6200,NaN,gemini-2.0-flash-001,08_Dict_22,def solve():\n n = int(input())\n ice_cr...,NaN,NaN,2025-03-21 19:43:16.459842,generated
6201,NaN,gemini-2.0-flash-001,08_Dict_22,def solve():\n n = int(input())\n prices...,NaN,NaN,2025-03-21 19:43:18.512018,generated
6202,NaN,gemini-2.0-flash-001,08_Dict_22,def solve():\n n = int(input())\n prices...,NaN,NaN,2025-03-21 19:43:20.765874,generated
6203,NaN,gemini-2.0-flash-001,08_Dict_22,def solve():\n n = int(input())\n ice_cr...,NaN,NaN,2025-03-21 19:43:49.863853,generated


In [19]:
rag_submissions = submissions[submissions["type"].isin(["human", "generated"])]
rag_submissions.groupby(["problem_id", "type"]).size().unstack(fill_value=0)

type,generated,human
problem_id,,
05_List_11,62,2029
08_Dict_22,378,2018


In [20]:
from typing import Tuple

import pandas as pd
from sklearn.model_selection import train_test_split


def split_dataframe(
    df: pd.DataFrame, stratify_cols: list[str], random_state: int = 42
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Splits the DataFrame into train/val/test while preserving the distribution of stratify_cols.

    Args:
        df (pd.DataFrame): The DataFrame to split.
        stratify_cols (list[str]): Columns to stratify by (e.g., ["problem_id", "type"]).
        random_state (int): Random seed for reproducibility.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]
    """
    df["_stratify_group"] = df[stratify_cols].astype(str).agg("_".join, axis=1)

    train_df, temp_df = train_test_split(
        df, test_size=0.2, stratify=df["_stratify_group"], random_state=random_state
    )

    val_df, test_df = train_test_split(
        temp_df,
        test_size=0.5,
        stratify=temp_df["_stratify_group"],
        random_state=random_state,
    )

    for d in (train_df, val_df, test_df):
        d.drop(columns="_stratify_group", inplace=True)

    return train_df, val_df, test_df


In [21]:
# from split_df import split_dataframe

train_df, val_df, test_df = split_dataframe(rag_submissions, stratify_cols=["problem_id", "type"], random_state=42)

train_df.shape, val_df.shape, test_df.shape

<ipython-input-20-2c1ed5c03801>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["_stratify_group"] = df[stratify_cols].astype(str).agg("_".join, axis=1)


((3589, 8), (449, 8), (449, 8))

In [22]:
train_counts = train_df.groupby(["problem_id", "type"]).size().unstack(fill_value=0)
val_counts = val_df.groupby(["problem_id", "type"]).size().unstack(fill_value=0)
test_counts = test_df.groupby(["problem_id", "type"]).size().unstack(fill_value=0)

combined = pd.concat(
    [train_counts, val_counts, test_counts],
    axis=1,
    keys=["Train", "Validation", "Test"]
)

print(combined)

               Train       Validation            Test      
type       generated human  generated human generated human
problem_id                                                 
05_List_11        50  1623          6   203         6   203
08_Dict_22       302  1614         38   202        38   202


## Retrievers

## Local

In [ ]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_core.documents import Document

# code for indexing is in rag notebook
client = QdrantClient(host="localhost", port=6333)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="submission_collection",
    embedding=embeddings,
)

/var/folders/27/0_8pjdb52g91gmnzbtgblvgc0000gn/T/ipykernel_97571/1119695003.py:7: UserWarning: Qdrant client version 1.13.3 is incompatible with server version 1.9.2. Major versions should match and minor version difference must not exceed 1. Set check_version=False to skip version check.
  client = QdrantClient(host="localhost", port=6333)


## Colab GPUs

In [24]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(path="qdrant_storage")

if not client.collection_exists("submission_collection"):
    print("Creating collection...")
    client.create_collection(
        collection_name="submission_collection",
        vectors_config=VectorParams(size=768, distance=Distance.COSINE),
    )

vector_store = QdrantVectorStore(
    client=client,
    collection_name="submission_collection",
    embedding=embeddings,
)

Creating collection...


W0504 07:46:09.097000 301 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode


In [25]:
from langchain_core.documents import Document

all_docs = [
    Document(page_content=code, metadata={"problem_id": problem_id, "author": author, "type": _type})
    for code, problem_id, author, _type in zip(
        list(train_df["code"]),
        train_df["problem_id"],
        train_df["author"],
        train_df["type"])
]
print(len(all_docs))

3589


In [26]:
import math

batches = math.ceil(len(all_docs) / 10)
for i in tqdm(range(batches)):
    start = i * 10
    end = min((i + 1) * 10, len(all_docs))
    vector_store.add_documents(all_docs[start:end])

100%|██████████| 359/359 [02:15<00:00,  2.65it/s]


## Our retrievers

In [ ]:
retriever_normal = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 20},  # top_k 20
)

# Maximal Marginal Relevance (MMR) is a hybrid search
retriever_mmr = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 20, "lambda_mult": 0.5},
)

reranker = CrossEncoderReranker(model=model)
def reranked_retriever(query):
    # retrieve top 20 docs
    retrieved_docs = retriever_mmr.invoke(query)

    # rerank 20 docs, keep only top 3
    reranked_docs = reranker.compress_documents(retrieved_docs, query=query)[:3]

    return reranked_docs


query = generated_submissions.iloc[0]["code"]

print("Normal Retriever Results:")
res_normal = retriever_normal.invoke(query)
for i in range(3):
    print(res_normal[i].metadata)

print("\nMMR Retriever Results:")
res_mmr = retriever_mmr.invoke(query)
for i in range(3):
    print(res_mmr[i].metadata)

print("\nReranked Retriever Results:")
res_reranked = reranked_retriever(query)
for i in range(min(3, len(res_reranked))):
    print(res_reranked[i].metadata)

Normal Retriever Results:
{'problem_id': '05_List_11', 'author': 'gemini-2.0-flash-001', 'type': 'generated', '_id': '3f3024d42e4e4430b9c56cbb001e442f', '_collection_name': 'submission_collection'}
{'problem_id': '05_List_11', 'author': 'gemini-2.0-flash-001', 'type': 'generated', '_id': 'b389d504ba854350bb3aaeb6ada9697a', '_collection_name': 'submission_collection'}
{'problem_id': '05_List_11', 'author': 'gemini-2.0-flash-001', 'type': 'generated', '_id': '5f2083c4a38849e2a0b1da7889970d70', '_collection_name': 'submission_collection'}

MMR Retriever Results:
{'problem_id': '05_List_11', 'author': 'gemini-2.0-flash-001', 'type': 'generated', '_id': '3f3024d42e4e4430b9c56cbb001e442f', '_collection_name': 'submission_collection'}
{'problem_id': '05_List_11', 'author': '6330405021', 'type': 'human', '_id': '9f47f593bf04488e954cacd78aef4d7f', '_collection_name': 'submission_collection'}
{'problem_id': '05_List_11', 'author': 'gemini-2.0-flash-001', 'type': 'generated', '_id': '52038149f15e

In [28]:
generated_submissions.iloc[0]

,0
id,NaN
author,gemini-2.0-flash-001
problem_id,05_List_11
code,s = input()\nmissing = []\nfor i in range(10):...
submission_id,NaN
score,NaN
time,2025-04-28 22:43:32.473922
type,generated


## Retrieval Evaluation

Calculate the MRR of the two retrievers based on the top 3 returned by each retriever.

In [29]:
def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    dot_product = np.dot(vec1, vec2)

    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)

    return similarity

In [30]:
columns = [
        "submission_id", "code", "author", "type",
        "retrieved_code", "retrieved_author", "retrieved_type", "similarity"
    ]

retrieved_docs_normal = pd.DataFrame(columns=columns)
retrieved_docs_mrr = pd.DataFrame(columns=columns)
retrieved_docs_reranked = pd.DataFrame(columns=columns)

In [33]:
limit = -1
count = 0
hit_normal = miss_normal = 0

for _, row in tqdm(val_df.iterrows()):
    submission_id = row["submission_id"]
    code = row["code"]
    author = row["author"]
    _type = row["type"]

    res_normal = retriever_normal.invoke(query)
    similarities = []
    for i in range(3):
        retrieved_author = res_normal[i].metadata["author"]
        retrieved_code = res_normal[i].page_content
        retrieved_type = res_normal[i].metadata["type"]

        val_embed = embeddings.embed_query(code)
        doc_embed = embeddings.embed_query(retrieved_code)
        similarity = cosine_similarity(val_embed, doc_embed)
        similarities.append(similarity)

        retrieved_docs_normal.loc[len(retrieved_docs_normal)] = [
            submission_id, code, author, _type,
            retrieved_code, retrieved_author, retrieved_type, similarity]

    found = False
    for i in range(3):
        # print(f"Similarity {i}: {similarities[i]}")
        if similarities[i] > 0.8:
            hit_normal += (1/(i+1)) * similarity
            found = True
            break
    if not found:
        # print(f"Query {count}: Failed to find relevant doc in top 3 results")
        miss_normal += 1

    count += 1
    if count % 50 == 0:
        print(f"Processed {count} queries, hit: {hit_normal}, miss: {miss_normal}")
    if limit != -1 and count == limit:
        break

print(f"Hit NORMAL: {hit_normal}")
print(f"Miss NORMAL: {miss_normal}")

50it [00:13,  3.75it/s]

Processed 50 queries, hit: 17.474316241431225, miss: 30


100it [00:27,  3.80it/s]

Processed 100 queries, hit: 35.87828293490275, miss: 59


150it [00:41,  3.79it/s]

...

Processed 350 queries, hit: 129.57027671414463, miss: 200


400it [01:52,  3.58it/s]

Processed 400 queries, hit: 147.75266394931748, miss: 229


449it [02:05,  3.57it/s]

Hit NORMAL: 165.79637078612055
Miss NORMAL: 256


In [34]:
retrieved_docs_normal.to_csv("retrieved_docs_normal.csv", index=False)

In [35]:
limit = -1
count = 0
hit_mrr = miss_mrr = 0

for _, row in tqdm(val_df.iterrows()):
    submission_id = row["submission_id"]
    code = row["code"]
    author = row["author"]
    _type = row["type"]

    res_mmr = retriever_mmr.invoke(code)
    similarities = []
    for i in range(3):
        retrieved_author = res_mmr[i].metadata["author"]
        retrieved_code = res_mmr[i].page_content
        retrieved_type = res_mmr[i].metadata["type"]

        val_embed = embeddings.embed_query(code)
        doc_embed = embeddings.embed_query(retrieved_code)
        similarity = cosine_similarity(val_embed, doc_embed)
        similarities.append(similarity)

        retrieved_docs_mrr.loc[len(retrieved_docs_mrr)] = [
            submission_id, code, author, _type,
            retrieved_code, retrieved_author, retrieved_type, similarity]

    found = False
    for i in range(3):
        if similarities[i] > 0.8:
            hit_mrr += (1/(i+1)) * similarity
            found = True
            break
    if not found:
        # print(f"Query {count}: Failed to find relevant doc in top 3 results")
        miss_mrr += 1

    count += 1
    if count % 50 == 0:
        print(f"Processed {count} queries, hit: {hit_mrr}, miss: {miss_mrr}")
    if limit != -1 and count == limit:
        break

print(f"Hit MRR: {hit_mrr}")
print(f"Miss MRR: {miss_mrr}")

50it [00:17,  2.96it/s]

Processed 50 queries, hit: 47.63169301163396, miss: 0


100it [00:34,  2.33it/s]

Processed 100 queries, hit: 95.11844042157888, miss: 0


150it [00:51,  3.13it/s]

...

Processed 350 queries, hit: 332.5579133994123, miss: 0


400it [02:18,  3.12it/s]

Processed 400 queries, hit: 380.1304057686113, miss: 0


449it [02:34,  2.90it/s]

Hit MRR: 426.7760087864129
Miss MRR: 0


In [36]:
retrieved_docs_mrr.to_csv("retrieved_docs_mrr.csv", index=False)

In [31]:
limit = -1
count = 0
hit_reranked = miss_reranked = 0

for _, row in tqdm(val_df.iterrows()):
    submission_id = row["submission_id"]
    code = row["code"]
    author = row["author"]
    _type = row["type"]

    res_reranked = reranked_retriever(code)
    similarities = []
    for i in range(3):
        retrieved_author = res_reranked[i].metadata["author"]
        retrieved_code = res_reranked[i].page_content
        retrieved_type = res_reranked[i].metadata["type"]

        val_embed = embeddings.embed_query(code)
        doc_embed = embeddings.embed_query(retrieved_code)
        similarity = cosine_similarity(val_embed, doc_embed)
        similarities.append(similarity)

        retrieved_docs_reranked.loc[len(retrieved_docs_reranked)] = [
            submission_id, code, author, _type,
            retrieved_code, retrieved_author, retrieved_type, similarity]

    found = False
    for i in range(3):
        if similarities[i] > 0.8:
            hit_reranked += (1/(i+1)) * similarity
            found = True
            break
    if not found:
        # print(f"Query {count}: Failed to find relevant doc in top 3 results")
        miss_reranked += 1

    count += 1
    if count % 50 == 0:
        print(f"Processed {count} queries, hit: {hit_reranked}, miss: {miss_reranked}")
    if limit != -1 and count == limit:
        break

print(f"Hit RERANKED: {hit_reranked}")
print(f"Miss RERANKED: {miss_reranked}")

50it [01:04,  1.36s/it]

Processed 50 queries, hit: 47.68351088768879, miss: 0


100it [02:05,  1.48s/it]

Processed 100 queries, hit: 95.22387031537635, miss: 0


150it [03:04,  1.15it/s]

...

Processed 350 queries, hit: 332.8497281177052, miss: 0


400it [08:21,  1.49s/it]

Processed 400 queries, hit: 380.5032258120924, miss: 0


449it [09:23,  1.25s/it]

Hit RERANKED: 427.19602970720354
Miss RERANKED: 0


In [32]:
retrieved_docs_mrr.to_csv("retrieved_docs_reranked.csv", index=False)

In [38]:
sample_size = val_df.shape[0]

# this MRR = mean reciprocal rank
print(f"MRR of normal retriever: {hit_normal / sample_size:.4f}")
print(f"MRR of MRR retriever: {hit_mrr / sample_size:.4f}")
print(f"MRR of reranked retriever: {hit_reranked / sample_size:.4f}")

print(f"Percentage of queries with no relevant doc in top 3 (normal): {100 * miss_normal / sample_size:.2f}%")
print(f"Percentage of queries with no relevant doc in top 3 (MMR): {100 * miss_mrr / sample_size:.2f}%")
print(f"Percentage of queries with no relevant doc in top 3 (rerank): {100 * miss_reranked / sample_size:.2f}%")

MRR of normal retriever: 0.3693
MRR of MRR retriever: 0.9505
MRR of reranked retriever: 0.9514
Percentage of queries with no relevant doc in top 3 (normal): 57.02%
Percentage of queries with no relevant doc in top 3 (MMR): 0.00%
Percentage of queries with no relevant doc in top 3 (rerank): 0.00%
